In [1]:
# to generate result sheet autoly
import pandas as pd 
import numpy as np 
import scipy
# import xlrd 
import sklearn

from Gibbs_model_probit import Gibbs_sampling
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal
from utils import baseline_lr,baseline_esnet,baseline_justmean
from utils import baseline_LogitElsnet,baseline_justmode,baseline_random,baseline_LogitLR,baseline_RanForest,baseline_Gibbs_zhe,baseline_SVM
from utils import baseline_LogitElsnet_leaveone,baseline_LogitLR_leaveone,baseline_SVM_leaveone
from utils import logis_regression_5,kernel_svm_regression_5,random_forest_regression_5,gpr_regression_5,logis_lasso_regression_5
from sklearn.model_selection import KFold
from scipy.stats import binom 
from scipy.stats import norm
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from tqdm import trange
from Gibbd_zhe import GibbsSampling3
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import LeaveOneOut
import time
from sklearn.preprocessing import StandardScaler
# from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, Matern
from sklearn.linear_model import LogisticRegression

In [2]:
# data_loading 
np.random.seed(123)

# data_table = pd.read_csv('../data/processed/all_feature_p1_lip_total_single.csv')
data_table_cont = pd.read_csv('../data/processed/all_feature_p1_spicie_cont_single.csv')
data_table_binary = pd.read_csv('../data/processed/all_feature_p1_spicie_binary_single.csv')
data_table_binary_standard = pd.read_csv('../data/processed/all_feature_p1_spicie_binary_standard_single.csv')




In [3]:
# last seven columns are labels
all_feature_cont = data_table_cont.iloc[:,1:-7]
all_feature_binary = data_table_binary.iloc[:,1:-7]
all_feature_binary_standard = data_table_binary_standard.iloc[:,1:-7]
all_feature_binary

,lip_signature,lip_P1_Cer_Cer(d16:1/16:0)_high,lip_P1_Cer_Cer(d16:1/16:0)_low,lip_P1_Cer_Cer(d16:1/18:0)_high,lip_P1_Cer_Cer(d16:1/18:0)_low,lip_P1_Cer_Cer(d16:1/20:0)_high,lip_P1_Cer_Cer(d16:1/20:0)_low,lip_P1_Cer_Cer(d16:1/22:0)_high,lip_P1_Cer_Cer(d16:1/22:0)_low,lip_P1_Cer_Cer(d16:1/23:0)_high,...,lip_P1_TG(O) [NL]_TG(O-54:3) [NL-18:1]_high,lip_P1_TG(O) [NL]_TG(O-54:3) [NL-18:1]_low,lip_P1_TG(O) [NL]_TG(O-54:4) [NL-17:1]_high,lip_P1_TG(O) [NL]_TG(O-54:4) [NL-17:1]_low,lip_P1_TG(O) [NL]_TG(O-54:4) [NL-18:2]_high,lip_P1_TG(O) [NL]_TG(O-54:4) [NL-18:2]_low,gene_P1_ATM_SNP,gene_P1_BRCA1_SNP,gene_P1_BRCA2_SNP,gene_P1_CHEK2_SNP
0,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,0,0,0
1,1,1,0,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,0,0,0
2,1,0,1,1,0,1,0,0,1,0,...,1,0,1,0,0,1,0,0,1,0
3,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,1,0,0,0,0,1
4,0,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0,1,0,1,0,1,0,1,0,1,...,1,0,1,0,1,0,0,0,0,0
67,1,0,1,0,1,0,1,0,1,0,...,1,0,0,1,0,1,0,0,0,0
68,0,1,0,1,0,1,0,1,0,1,...,1,0,1,0,1,0,0,0,0,0
69,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,0,0,0


In [4]:
# P1(mHSPC) testtosterone suppression failure: CER(ceramide), DG(diacylglycerol), TG(triaclyglycerol)
# P2(mCRPC) overall survival:  CER(ceramide), Sph(Sphingosine), Acylcarnitine(acylcarnitine)
def feature_filter(df):
    all_feature = list(df)
    feature_dict = {}
    feature_dict['gene'] = [item for item in all_feature if 'gene_' in item]
    # feature_dict['TG'] = [item for item in all_feature if 'lip_P1' in item and 'lip_P1_TG' not in item] # remove the TG lip
    # lip_feature_P2 = [item for item in all_feature if 'lip_P2' in item and '_Cer' not in item]
    # lip_feature_all =  [item for item in all_feature if 'lip_' in item and 'P1_Cer' not in item]
    

    feature_dict['TG'] = [item for item in all_feature if 'lip_P1_TG' in item]
    feature_dict['Cer'] = [item for item in all_feature if 'lip_P1_Cer' in item]
    feature_dict['DG'] = [item for item in all_feature if 'lip_P1_DG' in item]

    feature_dict['Acy'] = [item for item in all_feature if '_Acylcarnitine' in item]
    feature_dict['Sph'] = [item for item in all_feature if '_Sph' in item]
    feature_dict['sig'] = [item for item in all_feature if 'signature' in item]

    return feature_dict

In [5]:
# gene_feature,lip_feature_P1,lip_feature_P2,lip_feature_all,\
#     lip_feature_TG,lip_feature_Cer,lip_feature_DG,\
#         lip_feature_Acylcarnitine,lip_feature_Sph,lip_feature_sig = feature_filter(data_table_binary)

# data_table = data_table_cont
data_table = data_table_binary
# data_table = data_table_binary_standard


feature_dict = feature_filter(data_table)
one_feature_P1 = [['TG'],['Cer'],['DG']]
one_feature_P2 = [['Cer'],['Acy'],['Sph']]

two_feature_P1 = [['TG','Cer'],['TG','DG'],['DG','Cer']]
two_feature_P2 = [['Acy','Cer'],['Sph','Cer'],['Sph','Acy']]

three_feature_P1 = [['TG','Cer','DG']]
three_feature_P2 = [['Cer','Acy','Sph']]


In [1]:
index = ['logistic reg','logistic reg-lasso','linear-kernel svm','RBF-kernel svm','RBF-kernel GPR','matern-kernel GPR' ]
result_table = pd.DataFrame(index = index)



# target ='1= death; 0=alive'
# target ='gap_surv_time_class'
# sub_target = '_early_death'
# sub_target = '_long_survive'

# target = 'ADT_if_fail'

Y = data_table_cont[target].values

if target == 'gap_surv_time_class':
    # for this target, Y=0 means early death, Y=1 means long survival, Y=2 for the middle value
    if sub_target == '_early_death':
        Y = (Y==0)*1 # early death predict
        target = target + sub_target
    elif sub_target == '_long_survive':
        Y = (Y==1)*1 # long survive predict
        target = target + sub_target

elif target == 'ADT_if_fail':
    # for this target, Y=0 means ADT success(not fail), otherwise ADT failure
    Y = (Y==0)*1

NameError: name 'pd' is not defined

In [54]:
Y.sum()

38

In [35]:
# single test gene:


kernel_white = DotProduct() + WhiteKernel()
kernel_RBF = RBF()
kernel_matern = Matern()


# target = 'ADT_if_fail'#'1= death; 0=alive'#'gap_surv_time_class' 'ADT_if_fail'
# Y = data_table_cont[target].values
# Y = (Y==0)*1


X = data_table[feature_dict['gene']].values
log_loss = logis_regression_5(X,Y)
log_lasso_loss = logis_lasso_regression_5(X,Y,0.1)
svm_loss_rbf = kernel_svm_regression_5(X,Y,'rbf')
svm_loss_linear = kernel_svm_regression_5(X,Y,'linear')

# gpr_white_loss = gpr_regression_5(X,Y,kernel_white)
gpr_RBF_loss = gpr_regression_5(X,Y,kernel_RBF)
gpr_matern_loss = gpr_regression_5(X,Y,kernel_matern)

loss = [log_loss,log_lasso_loss,svm_loss_linear,svm_loss_rbf,gpr_RBF_loss,gpr_matern_loss]

result_table['gene'] = loss

G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\gaussian_process\_gpr.py:506: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\gaussian_process\_gpr.py:506: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("

In [1]:
# single sig gene:


# X = data_table[feature_dict['sig']].values
# log_loss = logis_regression_5(X,Y)
# log_lasso_loss = logis_lasso_regression_5(X,Y,0.3)

# svm_loss = kernel_svm_regression_5(X,Y)
# gpr_white_loss = gpr_regression_5(X,Y,kernel_white)
# gpr_RBF_loss = gpr_regression_5(X,Y,kernel_RBF)
# gpr_matern_loss = gpr_regression_5(X,Y,kernel_matern)
# print(log_loss,log_lasso_loss,svm_loss,gpr_white_loss,gpr_RBF_loss,gpr_matern_loss)

# print(Y.sum())

In [36]:
# target = '1= death; 0=alive'#'gap_surv_time_class' 'ADT_if_fail'
# target = '1= death; 0=alive'#'gap_surv_time_class' 'ADT_if_fail'
# Y = data_table_cont[target].values

def all_model(data_table,result_table,features,feature_dict,Y,add_gene=False):


    for fes in features:
        
        feature_name = '_'.join(fes)
        feature_list = [data_table[feature_dict[item]].values for item in fes]

        if add_gene:
            feature_name = 'gene_'+feature_name
            feature_list.append(data_table[feature_dict['gene']].values)

        X = np.concatenate(feature_list,1)
        log_loss = logis_regression_5(X,Y)
        log_lasso_loss = logis_lasso_regression_5(X,Y,0.1)
        svm_loss_rbf = kernel_svm_regression_5(X,Y,'rbf')
        svm_loss_linear = kernel_svm_regression_5(X,Y,'linear')

        gpr_RBF_loss = gpr_regression_5(X,Y,kernel_RBF)
        gpr_matern_loss = gpr_regression_5(X,Y,kernel_matern)

        loss = [log_loss,log_lasso_loss,svm_loss_linear,svm_loss_rbf,gpr_RBF_loss,gpr_matern_loss]
        
        result_table[feature_name] = loss

all_model(data_table=data_table,\
        result_table=result_table,\
        features=one_feature_P1,\
        feature_dict=feature_dict,Y=Y,\
        add_gene=False)
all_model(data_table=data_table,\
        result_table=result_table,\
        features=one_feature_P1,\
        feature_dict=feature_dict,Y=Y,\
        add_gene=True)

all_model(data_table=data_table,\
        result_table=result_table,\
        features=two_feature_P1,\
        feature_dict=feature_dict,Y=Y,\
        add_gene=False)
all_model(data_table=data_table,\
        result_table=result_table,\
        features=two_feature_P1,\
        feature_dict=feature_dict,Y=Y,\
        add_gene=True)   
          
all_model(data_table=data_table,\
        result_table=result_table,\
        features=three_feature_P1,\
        feature_dict=feature_dict,Y=Y,\
        add_gene=False)
all_model(data_table=data_table,\
        result_table=result_table,\
        features=three_feature_P1,\
        feature_dict=feature_dict,Y=Y,\
        add_gene=True)


G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef

In [37]:
result_table

,gene,TG,Cer,DG,gene_TG,gene_Cer,gene_DG,TG_Cer,TG_DG,DG_Cer,gene_TG_Cer,gene_TG_DG,gene_DG_Cer,TG_Cer_DG,gene_TG_Cer_DG
logistic reg,0.620000,0.465714,0.479048,0.604762,0.465714,0.493333,0.619048,0.480952,0.520952,0.533333,0.466667,0.520952,0.519048,0.480952,0.480952
logistic reg-lasso,0.620000,0.451429,0.506667,0.590476,0.451429,0.507619,0.619048,0.509524,0.505714,0.519048,0.509524,0.505714,0.505714,0.508571,0.508571
linear-kernel svm,0.620000,0.493333,0.504762,0.564762,0.480000,0.506667,0.589524,0.480952,0.536190,0.545714,0.480000,0.493333,0.560952,0.495238,0.507619
RBF-kernel svm,0.592381,0.520000,0.489524,0.534286,0.520000,0.504762,0.548571,0.534286,0.504762,0.560952,0.534286,0.518095,0.546667,0.519048,0.533333
RBF-kernel GPR,0.605714,0.451429,0.548571,0.494286,0.451429,0.534286,0.521905,0.464762,0.479048,0.531429,0.464762,0.479048,0.544762,0.464762,0.464762
matern-kernel GPR,0.605714,0.465714,0.549524,0.480000,0.465714,0.547619,0.493333,0.424762,0.479048,0.560000,0.424762,0.493333,0.519048,0.422857,0.422857


In [109]:
file_name = './result_log/P1_pred_'+target+'.csv'
result_table.to_csv(file_name)

In [39]:
# weight analysis 

def logis_regression_weight_analysis(X,Y,l1_ratio=0.5):
    
    kf = KFold(n_splits=5)
    kf.get_n_splits(X)

    clf_list = []

    best_index = 0
    best_score = 0
    loss = []
    
    for i,index in enumerate (kf.split(X)):
        train_index, test_index = index
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        clf = LogisticRegression(random_state=0,penalty='elasticnet',solver='saga',l1_ratio=l1_ratio
).fit(X_train, y_train)
        clf_list.append(clf)

        test_score = clf.score(X_test,y_test)
        # print(test_score)
        loss.append(test_score)

        if test_score>best_score:
            best_index = i
            best_score = test_score
    

    return np.array(loss).mean(),clf_list[best_index]


def get_sparse_name(clf,feature_name,top=30):
    weight = clf.coef_.squeeze()
    
    weight_table = pd.DataFrame()
    weight_table['feature_name'] = feature_name
    weight_table['feature_weight'] = clf.coef_.squeeze()
    weight_table_de  = weight_table.sort_values('feature_weight',ignore_index=False).iloc[0:top]
    weight_table_in  = weight_table.sort_values('feature_weight',ascending=False,ignore_index=False).iloc[0:top]

    return weight_table_de,weight_table_in

# fes = ['DG','Cer','gene'] # optimal feature groups for survial prediction 
# fes = ['DG','gene'] # optimal feature groups for survial prediction 

fes = ['DG','gene']# optimal feature groups for ADT-fail prediction
# fes = ['TG','Cer','gene']# optimal feature groups for early-dead prediction

feature_name = [name for item in fes for name in feature_dict[item]]

feature_list = [data_table[feature_dict[item]].values for item in fes]
X = np.concatenate(feature_list,1)

loss, clf=logis_regression_weight_analysis(X,Y,0.1)
print('loss',loss)
weight_table_de,weight_table_in = get_sparse_name(clf,feature_name,top=30)


loss 0.619047619047619


In [40]:
# def table_process():

def str_clean(string):
    split_list = string.split('_')
    new_list = [split_list[0]]+split_list[2:]
    return '_'.join(new_list)

def effect_ratio(table):
    table['relative_effects_ratio'] = table['feature_weight']/table['feature_weight'].sum()

def table_process(table):

    table[['feature_name']] = table[['feature_name']].applymap(str_clean)
    effect_ratio(table)

table_process(weight_table_de)
table_process(weight_table_in)


In [43]:
file_name_de = './result_log/P1_weight_'+target+'_de_'+'.csv'
file_name_in = './result_log/P1_weight_'+target+'_in_'+'.csv'
weight_table_de.to_csv(file_name_de)
weight_table_in.to_csv(file_name_in)


In [119]:
def sig(x):
    return 1/(1+np.exp(-x))
sig(-1.12)

0.2460112835510519

In [41]:
weight_table_in

,feature_name,feature_weight,relative_effects_ratio
39,lip_DG_DG(16:0_22:6)_low,0.879734,0.114514
36,lip_DG_DG(18:2_20:4)_high,0.638440,0.083105
20,lip_DG_DG(18:1_18:2)_high,0.631783,0.082238
10,lip_DG_DG(16:1_18:1)_high,0.548243,0.071364
31,lip_DG_DG(18:0_20:4)_low,0.428050,0.055719
28,lip_DG_DG(18:1_20:3)_high,0.415016,0.054022
17,lip_DG_DG(18:1_18:1)_low,0.395717,0.051510
13,lip_DG_DG(16:0_18:2)_low,0.391659,0.050982
6,lip_DG_DG(14:0_18:2)_high,0.390049,0.050772
48,lip_DG_DG(18:2_22:6)_high,0.369094,0.048044


In [42]:
weight_table_de

,feature_name,feature_weight,relative_effects_ratio
53,gene_CHEK2_SNP,-1.126880,0.131903
38,lip_DG_DG(16:0_22:6)_high,-0.851522,0.099672
50,gene_ATM_SNP,-0.726550,0.085044
37,lip_DG_DG(18:2_20:4)_low,-0.610214,0.071427
21,lip_DG_DG(18:1_18:2)_low,-0.603555,0.070647
11,lip_DG_DG(16:1_18:1)_low,-0.520048,0.060873
30,lip_DG_DG(18:0_20:4)_high,-0.399813,0.046799
29,lip_DG_DG(18:1_20:3)_low,-0.386791,0.045275
16,lip_DG_DG(18:1_18:1)_high,-0.367487,0.043015
12,lip_DG_DG(16:0_18:2)_high,-0.363445,0.042542
